In [ ]:
import tweepy
import pandas as pd
import re, string, random
from nltk.tag import pos_tag
from nltk import sent_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize
from nltk.corpus import sentiwordnet as swn
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import twitter_samples, stopwords
from nltk import FreqDist, classify, NaiveBayesClassifier


#Global Database to store words their polarity score and frequencies
word_freq = pd.DataFrame(columns=['word','score','freq'])

#Consumer Key for Twitter API access
consumer_key = "wvSK124NAIimBMX0aY9gJ1O9H" 
consumer_secret = "lTlgZPfSSUQk2ZZkORex1asQzTL7KqOlgOJd6fj3BBTmVshoTq"
access_key = "1074517762960052224-m7DqeZhrjH77nKX3cJMzMTMTDRoIuy"
access_secret = "F6l0PtL4ub5WYu4YpT29MBKQbIRy9bqPmAXjKkFeK4q4i"

#Extract tweet from twitter API - Tweepy
def get_tweets(username): 
          
        # Authorization to consumer key and consumer secret 
        auth = tweepy.OAuthHandler(consumer_key, consumer_secret) 
  
        # Access to user's access key and access secret 
        auth.set_access_token(access_key, access_secret) 
  
        # Calling api 
        api = tweepy.API(auth) 
  
        # 10 tweets to be extracted 
        number_of_tweets=10
        tweets = api.user_timeline(screen_name=username) 
  
        # Empty Array 
        tmp=[]  
  
        # create array of tweet information: username,  
        # tweet id, date/time, text 
        tweets_for_csv = [tweet.text for tweet in tweets]
        return tweets_for_csv

#Data preprocessing for Naive Bayes Classifier
#removing unwanted data
def remove_noise(tweet_tokens, stop_words = ()):

    #empty list for cleaned tokens
    cleaned_tokens = []

    #check for token and tag in the tweet_tokens
    for token, tag in pos_tag(tweet_tokens):
        
        #remove @<string> and URL
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)
        
        #Add part of speech tag
        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        #Lemmatizing the tokens according to words present in WordNet corpus 
        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        #If token is not a punctuation and not in stop words add it to list
        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
            
    return cleaned_tokens

#creating a datamodel for Naive-Bayes Classifier
def get_tweets_for_model(cleaned_tokens_list):
    
    #Create a dictionary with token as key and "True" as its value
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

    
#Preprocessing data for sentiment score calculation using SentiWordNet
#Removing the unwanted data
def clean_text(text):
    text = text.replace("<br />", " ")
    text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', text)
    text = re.sub("(@[A-Za-z0-9_]+)","", text)
    text = re.sub("#","",text)
 
    return text

#Convert between the PennTreebank tags to simple Wordnet tags
def penn_to_wn(tag):
    
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None

#Storing the scores and frequency of words according to sentiwordnet
def sentiment_scores(text): 
    
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    
    tokens_count = 0
    global word_freq
    
    text = clean_text(text)
 
 
    raw_sentences = sent_tokenize(text)
    
    for raw_sentence in raw_sentences:
        
        tagged_sentence = pos_tag(word_tokenize(raw_sentence))
        print(tagged_sentence)

        for word, tag in tagged_sentence:
            
            wn_tag = penn_to_wn(tag)
            print("Word = ",word)
            print("Tag = ",wn_tag)
            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV, wn.VERB):
                continue
                
            word = stemmer.stem(word)
            print(word)
            
            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            print(lemma)
            
            if not lemma:
                continue

            synsets = wn.synsets(lemma, pos=wn_tag)
            if not synsets:
                continue

            # Take the first sense, the most common
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())
            print("SWN = ",swn_synset)

            if swn_synset.pos_score() < swn_synset.neg_score():
                if word not in word_freq.values:
                    word_freq = word_freq.append(pd.DataFrame({"word":[word],"score":[swn_synset.neg_score()],"freq":[1]}), ignore_index = True)

                else:
                    word_freq.loc[word_freq['word']==word,'freq'] +=1

            tokens_count += 1
            
    if not tokens_count:
        return 0
    
    
        
if __name__ == "__main__":

    #Datasample for training of Naive Bayes classifier
    positive_tweets = twitter_samples.strings('positive_tweets.json')
    negative_tweets = twitter_samples.strings('negative_tweets.json')
    
    #Selection of english stopwords
    stop_words = stopwords.words('english')

    #Tokenizing the data
    positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
    negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')

    positive_cleaned_tokens_list = []
    negative_cleaned_tokens_list = []

    #removing noise from the token lists
    for tokens in positive_tweet_tokens:
        positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

    for tokens in negative_tweet_tokens:
        negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

    #Create a model for classifier in form of a dictionary with each token as key and value as true
    #for both negative and positive token list
    positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
    negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)

    #Create dataset with each token list dictionary and adding positive for element in positive model
    #and negative for element in negative model
    positive_dataset = [(tweet_dict, "Positive")
                         for tweet_dict in positive_tokens_for_model]

    negative_dataset = [(tweet_dict, "Negative")
                         for tweet_dict in negative_tokens_for_model]

    dataset = positive_dataset + negative_dataset

    #randomly shuffles the dataset each time executed
    random.shuffle(dataset)

    #dividing the dataset into training and testing
    train_data = dataset[:7000]
    test_data = dataset[7000:]
    
    #Training the classifier and checking the accuracy score
    classifier = NaiveBayesClassifier.train(train_data)
    print("Accuracy is:", classify.accuracy(classifier, test_data))
    
    #Exctracting tweets of a user
    tweets = get_tweets("@realDonaldTrump")
    
    #Checking for each tweet
    for custom_tweet in tweets:
        
        #Removing noise
        custom_tokens = remove_noise(word_tokenize(custom_tweet))
        
        #use Naive Bayes classifier
        sentiment = classifier.classify(dict([token, True] for token in custom_tokens))
        
        #If sentiment is negative classify using SentiWordNet
        if(sentiment == 'Negative'):
            
            #Storing data for custom tweet
            sentiment_scores(custom_tweet)
            
            sentiment_freq = 0.0
            total_word_freq = 0.0
            word_freq.dropna()
            
            #Using mathematical equation to calculate overall score.
            for word in word_freq.word:
                sentiment_freq += (word_freq.loc[word_freq['word']==word,'score'].item() * word_freq.loc[word_freq['word']==word,'freq']).item()
                total_word_freq += word_freq.loc[word_freq['word']==word,'freq'].item()

            if sentiment_freq == 0 or total_word_freq == 0:
                print("neutral")
            else:    
                sentiment_score = sentiment_freq / total_word_freq

                print(sentiment_score)

                if sentiment_score >= 0.7:
                    print("suicidal")
                elif sentiment_score < 0.7 and sentiment_score >= 0.5:
                    print("depressed")
                else:
                    print("neutral")